In [28]:
import pandas as pd
import numpy as np
from scipy import stats 
from scipy.stats import zscore

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly

import seaborn as sns

import ipywidgets

import json
import os
import sys
import io
import datetime

# Start by mapping 'games' data in original raw format into dataframe
Manually extracting the data from the website has the data in a 1 column list repeating every 4 rows.
1. Date
2. (team a) vs (Team b) - in Home vs Away order
3. (score) to (score)
4. game_id



In [107]:
#df_games = pd.DataFrame(columns = ['GAME_ID', 'DATE', 'SCORE_STR', 'TEAMS_STR'])
df_games = pd.DataFrame(columns = ['GAME_ID', 'DATE', 'SCORE_STR', 'TEAMS_STR', 'TOWN_TEAM_HOME', 'TOWN_TEAM_AWAY', 'GOALS_HOME', 'GOALS_AWAY', 'PTS_HOME', 'PTS_AWAY'])
df_games

fp_games = ['../data/Grade 6 Girls Soccer - G6G_FALL2022_SCORES_RAW.csv', 
            '../data/Grade 6 Girls Soccer - G6G_SPRING2022_SCORES_RAW.csv']

for fp in fp_games:
    print(fp)
    file1 = open(fp, 'r')
    lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in lines:
        count += 1
        lineStr = line.strip()
        #print(lineStr)
        if (count % 4) == 1:
            d_dateStr = lineStr
            #print(d_dateStr)
            d_date = datetime.datetime.strptime(d_dateStr, '%m/%d/%Y')
            #d_date = d_dateStr
        elif (count % 4) == 2:
            d_teams = lineStr
            teamsArr = d_teams.split('vs.')
            d_team_home = teamsArr[0].strip()
            d_team_away = teamsArr[1].strip()
        elif (count % 4) == 3:
            d_scores = lineStr
            #win = 3, tie = 1, loss = 0. Forfeit (-3 forfeiting team, 0 other); double forfeit (both -3).
            if ('forfeit' not in d_scores.lower()):
                goalsArr = d_scores.split('to')
                d_goals_home = int(goalsArr[0])
                d_goals_away = int(goalsArr[1])
                
                if (d_goals_home == d_goals_away):
                    d_pts_home = 1
                    d_pts_away = 1
                elif (d_goals_home > d_goals_away):
                    d_pts_home = 3
                    d_pts_away = 0
                else:
                    d_pts_home = 0
                    d_pts_away = 3
            elif ('away forfeit' in d_scores.lower()):
                d_pts_home = 0
                d_pts_away = -3
            elif ('home forfeit' in d_scores.lower()):
                d_pts_home = -3
                d_pts_away = 0
            elif ('double forfeit' in d_scores.lower()):
                d_pts_home = -3
                d_pts_away = -3
            else:
                print(f'SCORE ERROR({count}): {d_scores}')
            
            
        elif (count % 4) == 0:
            d_gameid = lineStr
            #df_games.append({'GAME_ID' : d_gameid, 'DATE' : d_date, 'SCORE_STR' : d_scores, 'TEAMS_STR' : d_teams}, ignore_index = True) 
            df_games.loc[len(df_games.index)] = [d_gameid, d_date, d_scores, d_teams, d_team_home, d_team_away, d_goals_home, d_goals_away, d_pts_home, d_pts_away]
    
    file1.close()

# Convert dataframe columns to desired types
#df_games['DATE']= pd.to_datetime(df_games['DATE'])
df_games['DATE'] = df_games['DATE'].astype('datetime64[ns]')
df_games['GOALS_HOME'] = df_games['GOALS_HOME'].astype('int')
df_games['GOALS_AWAY'] = df_games['GOALS_AWAY'].astype('int')
df_games['PTS_HOME'] = df_games['PTS_HOME'].astype('int')
df_games['PTS_AWAY'] = df_games['PTS_AWAY'].astype('int')


# Map date to determine spring vs fall and create numeric representation for first half spring vs second half fall.
#df_games['YEAR_HALF'] = df_games.apply(lambda x: x.timetuple().tm_yday)
#df_games['YEAR_HALF'] = df['date'].dt.year + df['date'].dt.month.gt(6).add(1).astype(str)
df_games['YEAR_HALF'] = np.where(df_games['DATE'].dt.month.le(6), df_games['DATE'].dt.year + 0, df_games['DATE'].dt.year + 0.5)


df_games


../data/Grade 6 Girls Soccer - G6G_FALL2022_SCORES_RAW.csv
../data/Grade 6 Girls Soccer - G6G_SPRING2022_SCORES_RAW.csv


,GAME_ID,DATE,SCORE_STR,TEAMS_STR,TOWN_TEAM_HOME,TOWN_TEAM_AWAY,GOALS_HOME,GOALS_AWAY,PTS_HOME,PTS_AWAY,YEAR_HALF
0,G6GM1G1-28469,2022-09-10,2 to 1,Andover Dunn vs. Peabody Lasers,Andover Dunn,Peabody Lasers,2,1,3,0,2022.5
1,G6GM1G1-28459,2022-09-17,0 to 1,Marblehead MILITIA vs. Andover Dunn,Marblehead MILITIA,Andover Dunn,0,1,0,3,2022.5
2,G6GM1G1-28458,2022-09-24,4 to 1,Newburyport Blizzard vs. Andover Dunn,Newburyport Blizzard,Andover Dunn,4,1,3,0,2022.5
3,G6GM1G1-28464,2022-10-01,5 to 3,Andover Dunn vs. Topsfield Masco Vipers,Andover Dunn,Topsfield Masco Vipers,5,3,3,0,2022.5
4,G6GM1G1-28473,2022-10-15,3 to 1,Andover Dunn vs. Newburyport Blizzard,Andover Dunn,Newburyport Blizzard,3,1,3,0,2022.5
...,...,...,...,...,...,...,...,...,...,...,...
1445,S22G6GC2G3-26473,2022-04-30,1 to 4,Triton HURRICANES vs. Andover Rapinoe,Triton HURRICANES,Andover Rapinoe,1,4,0,3,2022.0
1446,S22G6GC2G3-26467,2022-05-07,3 to 5,Andover Rapinoe vs. Beverly Lynx,Andover Rapinoe,Beverly Lynx,3,5,0,3,2022.0
1447,S22G6GC2G3-26483,2022-05-14,1 to 6,Andover Rapinoe vs. Newburyport Storm,Andover Rapinoe,Newburyport Storm,1,6,0,3,2022.0
1448,S22G6GC2G3-26481,2022-05-21,0 to 2,Andover Rapinoe vs. Marblehead ARSENAL,Andover Rapinoe,Marblehead ARSENAL,0,2,0,3,2022.0


In [103]:

# Process string of <town> <team> into columns
uh = df_games['TOWN_TEAM_HOME'].unique()
df_teams = pd.DataFrame({'TOWN_TEAM' : uh})
df_teams = df_teams.sort_values(by='TOWN_TEAM')
#df_teams['TOWN_TEAM'].to_csv('../data/teams_export.csv', index=False)

# Add mapping of town to teams
df_towns = pd.read_csv('../data/towns.csv', header=None, names=['TOWN'])
df_towns = df_towns.sort_values(by='TOWN', key=lambda x: x.str.len(), ascending=False)
df_towns
for town in df_towns['TOWN']:
    i = len(town)
    #print(i, town)
    mask = df_teams['TOWN_TEAM'].str.startswith(town)
    df_teams.loc[mask, 'TOWN'] = town
    df_teams.loc[mask, 'TEAM'] = df_teams.TOWN_TEAM.apply(lambda x: x[i:])

df_teams.to_csv('../data/teams_export.csv', index=False)

df_teams


,TOWN_TEAM,TOWN,TEAM
102,Amesbury Gators,Amesbury,Gators
67,Amesbury STORM,Amesbury,STORM
127,Amesbury Storm,Amesbury,Storm
14,Andover Akers,Andover,Akers
86,Andover Arsenal,Andover,Arsenal
...,...,...,...
114,Winthrop Seasiders,Winthrop,Seasiders
82,Winthrop Waves,Winthrop,Waves
51,Woburn Cobras,Woburn,Cobras
20,Woburn Inferno,Woburn,Inferno


# Create standading dataframe per-season